#Goal

This project focuses on finding the solution for a Mixed Integer Linear Programming (MILP) problem using Pyomo and CBC solver, to optimize scheduling for athletic events. This was inspired by the challenge of high school teams in Northern Wisconsin, having to travel over 100 miles for interconference play.

Using the eight interconference teams as inputs, we have nine different constraints to ensure fairness of play. Then using the objective of minimizing travel, the model generates a schedule for each team across two seasons. This approach is also highly transferable for logistics and supply chain fleet optimization problems.

#Set-Up

In [ ]:
#importing pyomo libraries
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *
import csv

#Optimization Model

In [ ]:
#defining teams and weeks
teams = ["Northwestern", "St. Croix Falls", "Barron", "Ashland", "Hayward", "Bloomer", "Spooner", "Cumberland"]
num_teams = len(teams) # Eight teams
weeks = range(1, 8) # Seven weeks
seasons = [1, 2]  # Two seasons

#inputting our distance matrix for each team
distances = [
    [0, 67, 33,101, 24, 24, 34, 19],   # Northwestern
    [67, 0, 40,150, 90, 70,100, 85],   # St. Croix Falls
    [33,40, 0,120, 50, 20, 60, 45],    # Barron
    [101,150,120, 0, 80,110, 70,130],  # Ashland
    [24,90,50,80, 0,40,34,60],         # Hayward
    [24,70,20,110,40, 0,50,30],        # Bloomer
    [34,100,60,70,34,50, 0,55],        # Spooner
    [19,85,45,130,60,30,55, 0]         # Cumberland
]

model = ConcreteModel()

i_set = RangeSet(0, num_teams-1) # Away team
j_set = RangeSet(0, num_teams-1) # Home team
w_set = RangeSet(1, 7)           # Week of season
s_set = RangeSet(1, 2)           # Season

#decision variable: x[i,j,w,s] = 1 if team i plays at team j in week w during season s
model.x = Var(i_set, j_set, w_set, s_set, domain=Binary)

#decision variable: for tracking the travel distance for each team (i) during each season (s)
model.away_miles = Var(i_set, s_set, domain=NonNegativeReals)

#objective: minimize total travel distance across both seasons
model.obj = Objective(expr=sum(distances[i][j]*model.x[i,j,w,s] for i in i_set for j in j_set for w in w_set for s in s_set), sense=minimize)

#constraints
#1) no self-play
def no_self_play_rule(model, i, w, s):
    return model.x[i,i,w,s] == 0
model.no_self_play = Constraint(i_set, w_set, s_set, rule=no_self_play_rule)

#2) each team plays exactly 7 games per season
def games_per_team_rule(model, i, s):
    return sum(model.x[i,j,w,s] for j in j_set for w in w_set) + sum(model.x[j,i,w,s] for j in j_set for w in w_set) == 7
model.games_per_team = Constraint(i_set, s_set, rule=games_per_team_rule)

#3) one game per week per team per season
def one_game_per_week_rule(model, i, w, s):
    return sum(model.x[i,j,w,s] for j in j_set) + sum(model.x[j,i,w,s] for j in j_set) == 1
model.one_game_per_week = Constraint(i_set, w_set, s_set, rule=one_game_per_week_rule)

#4) home game limits (3 to 4 per season)
def min_home_games_rule(model, i, s):
    return sum(model.x[j,i,w,s] for j in j_set for w in w_set) >= 3
def max_home_games_rule(model, i, s):
    return sum(model.x[j,i,w,s] for j in j_set for w in w_set) <= 4
model.min_home_games = Constraint(i_set, s_set, rule=min_home_games_rule)
model.max_home_games = Constraint(i_set, s_set, rule=max_home_games_rule)

#5) no duplicate matchups within a season
def no_duplicate_matchups_rule(model, i, j, s):
    if i != j:
        return sum(model.x[i,j,w,s] + model.x[j,i,w,s] for w in w_set) <= 1
    return Constraint.Skip
model.no_duplicate_matchups = Constraint(i_set, j_set, s_set, rule=no_duplicate_matchups_rule)

#6) no more than 2 consecutive away games per season
def away_game_stretch_rule(model, i, w, s):
    if w <= 5:
        return sum(model.x[i,j,k,s] for j in j_set for k in range(w, w+3)) <= 2
    return Constraint.Skip
model.away_game_stretch = Constraint(i_set, w_set, s_set, rule=away_game_stretch_rule)

#7) no more than 2 consecutive home games per season
def home_game_stretch_rule(model, i, w, s):
    if w <= 5:
        return sum(model.x[j,i,k,s] for j in j_set for k in range(w, w+3)) <= 2
    return Constraint.Skip
model.home_game_stretch = Constraint(i_set, w_set, s_set, rule=home_game_stretch_rule)

#8) reciprocal home/away across seasons
def reciprocal_home_away_rule(model, i, j):
    if i != j:
        return sum(model.x[i,j,w,1] for w in w_set) == sum(model.x[j,i,w,2] for w in w_set)
    return Constraint.Skip
model.reciprocal_home_away = Constraint(i_set, j_set, rule=reciprocal_home_away_rule)

#9) the second seasons travel distance for each school, must be within 50 miles of the first seasons
def away_miles_definition_rule(model, i, s):
    return model.away_miles[i, s] == sum(
        distances[i][j] * model.x[i, j, w, s]
        for j in j_set for w in w_set)
model.away_miles_def = Constraint(i_set, s_set, rule=away_miles_definition_rule)

#enforcing the upper bound for constraint #9, to be within 50 miles
def balance_upper_rule(model, i):
    return model.away_miles[i, 1] - model.away_miles[i, 2] <= 50
model.balance_upper = Constraint(i_set, rule=balance_upper_rule)

#enforcing the lower bound for constraint #9, to be within 50 miles
def balance_lower_rule(model, i):
    return model.away_miles[i, 2] - model.away_miles[i, 1] <= 50
model.balance_lower = Constraint(i_set, rule=balance_lower_rule)

#solving using cbc with diagnostics
solver = SolverFactory('cbc')
solver.options['ratioGap'] = 0.01
solver.solve(model, tee=True)

#extracting schedule for both seasons
schedule = {s: {team: {"total_miles":0, "weeks":[]} for team in teams} for s in seasons}
total_travel = 0
for s in s_set:
    for i in i_set:
        for w in w_set:
            for j in j_set:
                if model.x[i,j,w,s].value == 1:
                    schedule[s][teams[i]]["total_miles"] += distances[i][j]
                    total_travel += distances[i][j]
                    schedule[s][teams[i]]["weeks"].append((w, f"at {teams[j]}"))
                if model.x[j,i,w,s].value == 1:
                    schedule[s][teams[i]]["weeks"].append((w, f"vs {teams[j]}"))

#finding the total travel miles for each season
season_total_travel = {}
for s in seasons:
    season_total_travel[s] = sum(schedule[s][team]["total_miles"] for team in teams)

#printing schedules
print(f"Heart of North Football total travel: {total_travel} miles\n\n")
for s in seasons:
    print(f"Season {s} travel distance: {season_total_travel[s]} miles")  # new summary line
    print(f"Season {s} schedule by team:\n")
    for team in teams:
        schedule[s][team]["weeks"].sort(key=lambda x: x[0])
        print(f"{team}, {schedule[s][team]['total_miles']} total travel miles")
        for w, game in schedule[s][team]["weeks"]:
            print(f"Week {w}: {game}")
        print()

#exporting to CSV
csv_filename = "football_schedule_two_seasons.csv"
with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    for s in seasons:
        writer.writerow([f"Season {s}"])
        for team, data in schedule[s].items():
            writer.writerow([f"{team}, {data['total_miles']} total travel miles"])
            for week, opponent in data["weeks"]:
                               writer.writerow([f"Week {week}:", opponent])
            writer.writerow([])

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Jun  7 2023 

command line - /content/bin/cbc -ratioGap 0.01 -printingOptions all -import /tmp/tmpkbf3gjqp.pyomo.lp -stat=1 -solve -solu /tmp/tmpkbf3gjqp.pyomo.soln (default strategy 1)
ratioGap was changed from 0 to 0.01
Option for printingOptions changed from normal to all
Presolve 464 (-168) rows, 800 (-112) columns and 10464 (-1824) elements
Statistics for presolved model
Original problem has 896 integers (896 of which binary)
Presolved problem has 784 integers (784 of which binary)
==== 16 zero objective 23 different
==== absolute objective values 23 different
==== for integers 0 zero objective 22 different
==== for integers absolute objective values 22 different
===== end objective counts


Problem has 464 rows, 800 columns (784 with objective) and 10464 elements
Column breakdown:
16 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of ty